### Kaggle Notebook

In [2]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [3]:
## 라이브러리 추가하기
import os
import sys
import random
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 버전문제로 모든 파일을 다운 후 설치 필요
sys.path.insert(0, '../input/timm-efficientdet-pytorch')
sys.path.insert(0, '../input/omegaconf')

# pytorch
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

# augmentation
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

# sci-kit learn
from sklearn.model_selection import StratifiedKFold

# etc
import time

from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet


import warnings
warnings.filterwarnings("ignore")


In [ ]:
df

In [21]:
df = pd.read_csv('./df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.drop(columns=['Unnamed: 0.1'], inplace=True)
df

,path,x,y,w,h
0,./data_pothole/ds1_511.jpg,1361.945996,953.269770,63.439024,77.530650
1,./data_pothole/ds1_511.jpg,987.311850,924.494335,89.149211,42.632578
2,./data_pothole/ds1_1845.jpg,1088.784657,792.205742,60.434724,27.698565
3,./data_pothole/ds1_1924.jpg,814.868102,837.851675,77.662180,16.502392
4,./data_pothole/ds1_1952.jpg,1375.621803,892.110048,138.819650,56.980861
...,...,...,...,...,...
4215,./data_pothole/ds4_G0348187.jpg,242.000000,1642.000000,156.000000,42.000000
4216,./data_pothole/ds4_G0398363.jpg,2674.000000,1722.000000,156.000000,44.000000
4217,./data_pothole/ds4_G0398363.jpg,1902.000000,1614.000000,118.000000,52.000000
4218,./data_pothole/ds4_G0398363.jpg,1898.000000,1550.000000,74.000000,30.000000


In [12]:
df['path'] = df['path'].apply(lambda x: '{}'.format(x.split('/')[2].strip()))
df

,path,x,y,w,h
0,ds1_511.jpg,1361.945996,953.269770,63.439024,77.530650
1,ds1_511.jpg,987.311850,924.494335,89.149211,42.632578
2,ds1_1845.jpg,1088.784657,792.205742,60.434724,27.698565
3,ds1_1924.jpg,814.868102,837.851675,77.662180,16.502392
4,ds1_1952.jpg,1375.621803,892.110048,138.819650,56.980861
...,...,...,...,...,...
4215,ds4_G0348187.jpg,242.000000,1642.000000,156.000000,42.000000
4216,ds4_G0398363.jpg,2674.000000,1722.000000,156.000000,44.000000
4217,ds4_G0398363.jpg,1902.000000,1614.000000,118.000000,52.000000
4218,ds4_G0398363.jpg,1898.000000,1550.000000,74.000000,30.000000


In [13]:
df['path'] = df['path'].apply(lambda x: '{}'.format(x.split('_')[1].strip()) if (x.split('_')[0].strip() == 'ds4') else x)
df

,path,x,y,w,h
0,ds1_511.jpg,1361.945996,953.269770,63.439024,77.530650
1,ds1_511.jpg,987.311850,924.494335,89.149211,42.632578
2,ds1_1845.jpg,1088.784657,792.205742,60.434724,27.698565
3,ds1_1924.jpg,814.868102,837.851675,77.662180,16.502392
4,ds1_1952.jpg,1375.621803,892.110048,138.819650,56.980861
...,...,...,...,...,...
4215,G0348187.jpg,242.000000,1642.000000,156.000000,42.000000
4216,G0398363.jpg,2674.000000,1722.000000,156.000000,44.000000
4217,G0398363.jpg,1902.000000,1614.000000,118.000000,52.000000
4218,G0398363.jpg,1898.000000,1550.000000,74.000000,30.000000


In [26]:
df = pd.read_csv('./df_final.csv')

In [27]:
df

,Unnamed: 0,path,x,y,w,h
0,0,ds1_511.jpg,1361.945996,953.269770,63.439024,77.530650
1,1,ds1_511.jpg,987.311850,924.494335,89.149211,42.632578
2,2,ds1_1845.jpg,1088.784657,792.205742,60.434724,27.698565
3,3,ds1_1924.jpg,814.868102,837.851675,77.662180,16.502392
4,4,ds1_1952.jpg,1375.621803,892.110048,138.819650,56.980861
...,...,...,...,...,...,...
4216,3891,G0348187.jpg,242.000000,1642.000000,156.000000,42.000000
4217,3892,G0398363.jpg,2674.000000,1722.000000,156.000000,44.000000
4218,3893,G0398363.jpg,1902.000000,1614.000000,118.000000,52.000000
4219,3894,G0398363.jpg,1898.000000,1550.000000,74.000000,30.000000


In [28]:
dir = '../input/pothole-end/train'
df['path'] = df['path'].apply(lambda x: '{}/{}'.format(dir,x) if (x.split('_')[0].strip() == 'ds1') else x)
df['path'] = df['path'].apply(lambda x: '{}/{}'.format(dir,x) if (x.split('_')[0].strip() == 'ds2') else x)
df['path'] = df['path'].apply(lambda x: '{}/{}'.format(dir,x) if (x.split('_')[0].strip() == 'ds3') else x)
df

,Unnamed: 0,path,x,y,w,h
0,0,../input/pothole-end/train/ds1_511.jpg,1361.945996,953.269770,63.439024,77.530650
1,1,../input/pothole-end/train/ds1_511.jpg,987.311850,924.494335,89.149211,42.632578
2,2,../input/pothole-end/train/ds1_1845.jpg,1088.784657,792.205742,60.434724,27.698565
3,3,../input/pothole-end/train/ds1_1924.jpg,814.868102,837.851675,77.662180,16.502392
4,4,../input/pothole-end/train/ds1_1952.jpg,1375.621803,892.110048,138.819650,56.980861
...,...,...,...,...,...,...
4216,3891,G0348187.jpg,242.000000,1642.000000,156.000000,42.000000
4217,3892,G0398363.jpg,2674.000000,1722.000000,156.000000,44.000000
4218,3893,G0398363.jpg,1902.000000,1614.000000,118.000000,52.000000
4219,3894,G0398363.jpg,1898.000000,1550.000000,74.000000,30.000000


In [30]:
dir = '../input/road-pothole-images-for-pothole-detection/Dataset 1 (Simplex)/Dataset 1 (Simplex)/Train data/Positive data'
df['path'] = df['path'].apply(lambda x: '{}/{}.JPG'.format(dir,x.split('.')[0].strip()) if x[0] == 'G' else x)
df

,Unnamed: 0,path,x,y,w,h
0,0,../input/pothole-end/train/ds1_511.jpg,1361.945996,953.269770,63.439024,77.530650
1,1,../input/pothole-end/train/ds1_511.jpg,987.311850,924.494335,89.149211,42.632578
2,2,../input/pothole-end/train/ds1_1845.jpg,1088.784657,792.205742,60.434724,27.698565
3,3,../input/pothole-end/train/ds1_1924.jpg,814.868102,837.851675,77.662180,16.502392
4,4,../input/pothole-end/train/ds1_1952.jpg,1375.621803,892.110048,138.819650,56.980861
...,...,...,...,...,...,...
4216,3891,../input/road-pothole-images-for-pothole-detec...,242.000000,1642.000000,156.000000,42.000000
4217,3892,../input/road-pothole-images-for-pothole-detec...,2674.000000,1722.000000,156.000000,44.000000
4218,3893,../input/road-pothole-images-for-pothole-detec...,1902.000000,1614.000000,118.000000,52.000000
4219,3894,../input/road-pothole-images-for-pothole-detec...,1898.000000,1550.000000,74.000000,30.000000


In [29]:
# dir = '../input/road-pothole-images-for-pothole-detection/Dataset 1 (Simplex)/Dataset 1 (Simplex)/Train data/Positive data'
# df['path'] = df['path'].apply(lambda x: '{}/{}.JPG'.format(dir,x.split('.')[0].strip()) 
#                               if (x.split('_')[1][0].strip() == 'G' and x.split('_')[0][2].strip() == '4') else x)
# df

IndexError: list index out of range

In [31]:
df.to_csv('df_kaggle.csv')

In [ ]:
## Configurations
class CFG:../input/road-pothole-images-for-pothole-detec...	
    mode = 'train'
    seed = 42
    print_freq = 30

    n_class = 13

    img_x = 512
    img_y = 512

    num_fold = 5
    num_epoch = 3
    batch_size = 8
    num_workers = 8    # decide how many data upload to dataset for a batch
                       # if n_workers 2, dataloader works twice for a batch.
                       # It has impact for cuda memory too

    lr = 0.0002

    max_grad_norm = 1000

    scheduler_params = dict(
        mode='min',
        factor=0.5,
        patience=1,
        verbose=False,
        threshold=0.0001,
        threshold_mode='abs',
        cooldown=0,
        min_lr=1e-8,
        eps=1e-08
    )

    data_dir = './data/images_2'
    ckpt_dir = './checkpoint'
    result_dir = './result'
    log_dir = './log'

    csv_dir = './data/train_dataset2.csv'
    csv2_dir = './data/train_dataset2_kfold.csv'

In [ ]:
##
def init_logger(path):
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler

    if not os.path.exists(path):
        os.makedirs(path)

    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=os.path.join(path, 'train_log'))
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger


##
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0    # val is value
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


##
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


## save model
def save_model(ckpt_dir, net, fold, num_epoch, epoch, batch, save_argument):
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    suffix = 'f{}_ep{}_bt{}_date{}.pth'.format(fold, epoch, batch, datetime.now().strftime("%m.%d-%H:%M"))

    filename = os.path.join(ckpt_dir, 'checkpoint_0.pth')
    best_filename = os.path.join(ckpt_dir, 'best_checkpoint_0.pth')
    final_filename = os.path.join(ckpt_dir, 'final_' + suffix)

    '''
    Effdet model structure has no first key name, model.
    
    EfficientDet(
        (backbone): EfficientNetFeatures(
            (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
            (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (act1): Swish()
            (blocks): Sequential(
                (0): Sequential(...
      
    resnext(
        (model): ResNet(
            (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act1): ReLU(inplace=True)
            (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
            (layer1): Sequential(
                (0): Bottleneck(...
    
    '''

    # save model every epoch
    # If you want to save model every epoch, change filename to suffix
    torch.save(net.model.state_dict(), filename)    # net.model.state_dict() makes no key, model.

    # leave only best model
    if save_argument:
        shutil.copyfile(filename, best_filename)

    if num_epoch == epoch:
        shutil.copyfile(best_filename, final_filename)
        os.remove(filename)
        os.remove(best_filename)


def load_model(ckpt_dir):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size = 512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(ckpt_dir)
    net.load_state_dict(checkpoint)

    net = DetBenchTrain(net, config)

    return net

In [ ]:
LOGGER = init_logger(CFG.log_dir)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
##
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, data_dir, transform=None):
        self.df = df
        self.data_dir = data_dir
        self.transform = transform

        self.arr_image = df['path'].values

    def __len__(self) -> int:    # -> is return function annotation
        return len(self.arr_image)

    def __getitem__(self, index):
        if random.random() > 0.5:
            image, boxes = self.load_image_and_boxes(index)
        else:
            image, boxes = self.load_cutmix_image_and_boxes(index)

        # label has only one class (background 0, object 1)
        # boxes.shape is (n_boxes, 4). So boxes.shape[0] would be n_boxes.
        labels = torch.ones((boxes.shape[0],), dtype=torch.int64)    # tensor([1, 1, ..., 1]) // num of 1 is n_boxes

        target = {}

        sample = image*255
        cv2.imwrite('a.jpg', sample)

        if self.transform:
            for i in range(10):
                # If image resize, boxes should resize also simultaneously.
                sample = {'image': image, 'bboxes': boxes, 'labels': labels}
                sample = self.transform(**sample)

                # if transform, some boxes would delete.
                # assert len(sample['bboxes']) == labels.shape[0], 'not equal!'

                # After Augmentation,
                # if image has no boxes, thus len(sample['bboxes'])=0 is True, loop again
                if len(sample['bboxes']) > 0:
                    image = sample['image']
                    # EfficientNet implementation in the effdet library takes bboxes in yxyx format
                    target['boxes'] = torch.tensor(sample['bboxes'])
                    target['boxes'][:, [0, 1, 2, 3]] = target['boxes'][:, [1, 0, 3, 2]]
                    target['labels'] = torch.stack(sample['labels']) # have to add this
                    break

        # save train sample
        sample = image.permute(1, 2, 0).cpu().numpy()
        sample = sample*255
        cv2.imwrite('train_sample.jpg', sample)

        return image, target


##
    def load_image_and_boxes(self, index):
        # input image
        image = cv2.imread(self.data_dir+self.arr_image[index], cv2.IMREAD_COLOR)
        # In open-cv, Image read is BGR order. and Albumentation use RGB format.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)

        # boxes
        # a bounding box formatted as a python-style (list of [xmin, ymin, width, height])
        path = self.arr_image[index]
        records = self.df[self.df['path'] == path ]
        boxes = records[['x', 'y', 'w', 'h']].values    # np array ([[..., ..., ..., ...], ...])

        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]    # xmax
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]    # ymax

        return image, boxes


##
    def load_cutmix_image_and_boxes(self, index, imsize=1024):
        """
        This implementation of cutmix author:  https://www.kaggle.com/nvnnghia
        Refactoring and adaptation: https://www.kaggle.com/shonenkov
        """

        w, h = imsize, imsize
        s = imsize // 2

        paths = self.df['path']
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]  # center x, y
        indexes = [index] + [random.randint(0, paths.shape[0] - 1) for _ in range(3)]

        result_image = np.full((imsize, imsize, 3), 1, dtype=np.float32)
        result_boxes = []

        for i, index in enumerate(indexes):
            image, boxes = self.load_image_and_boxes(index)

            if i == 0:
                x1a, y1a, x2a, y2a = max(xc - w, 0), max(yc - h, 0), xc, yc  # xmin, ymin, xmax, ymax (large image)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), h - (y2a - y1a), w, h  # xmin, ymin, xmax, ymax (small image)
            elif i == 1:  # top right
                x1a, y1a, x2a, y2a = xc, max(yc - h, 0), min(xc + w, s * 2), yc
                x1b, y1b, x2b, y2b = 0, h - (y2a - y1a), min(w, x2a - x1a), h
            elif i == 2:  # bottom left
                x1a, y1a, x2a, y2a = max(xc - w, 0), yc, xc, min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = w - (x2a - x1a), 0, max(xc, w), min(y2a - y1a, h)
            elif i == 3:  # bottom right
                x1a, y1a, x2a, y2a = xc, yc, min(xc + w, s * 2), min(s * 2, yc + h)
                x1b, y1b, x2b, y2b = 0, 0, min(w, x2a - x1a), min(y2a - y1a, h)
            result_image[y1a:y2a, x1a:x2a] = image[y1b:y2b, x1b:x2b]
            padw = x1a - x1b
            padh = y1a - y1b

            boxes[:, 0] += padw
            boxes[:, 1] += padh
            boxes[:, 2] += padw
            boxes[:, 3] += padh

            result_boxes.append(boxes)

        result_boxes = np.concatenate(result_boxes, 0)
        np.clip(result_boxes[:, 0:], 0, 2 * s, out=result_boxes[:, 0:])
        result_boxes = result_boxes.astype(np.int32)
        result_boxes = result_boxes[
            np.where((result_boxes[:, 2] - result_boxes[:, 0]) * (result_boxes[:, 3] - result_boxes[:, 1]) > 0)]


        # if BBox in an image is too small, upper method returns empty list. thus, no detected region. ex> car plate
        # So we need to change this.
        if len(result_boxes) == 0 :
            result_image, result_boxes = self.load_image_and_boxes(index)

        return result_image, result_boxes

In [ ]:
##
def transform_train():
    return A.Compose([A.RandomSizedCrop(min_max_height=(768, 768), height=1024, width=1024, p=0.5),
            A.OneOf([A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, val_shift_limit=0.2, p=0.9),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9)],p=0.9),
            A.ToGray(p=0.01),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Resize(height=512, width=512, p=1),
            A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)],
        p=1.0,
        bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0, label_fields=['labels']))

def transform_val():
    return A.Compose([A.Resize(height=512, width=512, p=1.0), ToTensorV2(p=1.0)],
                      A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        p=1.0,
        bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0, label_fields=['labels']))


In [ ]:
##
def train_one_epoch(loader_train, net, optim, epoch, device):
    batch_time = AverageMeter()
    losses = AverageMeter()

    net.train()
    start = end = time.time()

    for batch, (images, targets) in enumerate(loader_train, 1):
        images = torch.stack(images)
        images = images.to(device).float()
        boxes = [target['boxes'].to(device).float() for target in targets]
        labels = [target['labels'].to(device).float() for target in targets]

        optim.zero_grad()

        loss, _, _ = net(images, boxes, labels)

        # record loss
        batch_size = images.shape[0]
        losses.update(loss.detach().item(), batch_size)

        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=CFG.max_grad_norm)  # clipping grad

        optim.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if batch % CFG.print_freq == 0 or batch == len(loader_train):
            print('Epoch {0}: [{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f} (avg {loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                .format(
                epoch, batch, len(loader_train), batch_time=batch_time, loss=losses,
                remain=timeSince(start, float(batch) / len(loader_train)),
                grad_norm=grad_norm,
            ))

    return losses.avg

In [ ]:
##
def val_one_epoch(loader_val, net, device):
    batch_time = AverageMeter()
    losses = AverageMeter()

    net.eval()
    start = end = time.time()

    for batch, (images, targets) in enumerate(loader_val, 1):
        images = torch.stack(images)
        images = images.to(device).float()
        boxes = [target['boxes'].to(device).float() for target in targets]
        labels = [target['labels'].to(device).float() for target in targets]

        with torch.no_grad():
            loss, _, _ = net(images, boxes, labels)

        batch_size = images.shape[0]
        losses.update(loss.detach().item(), batch_size)

        batch_time.update(time.time() - end)
        end = time.time()

        if batch % CFG.print_freq == 0 or batch == (len(loader_val)):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f} (avg {loss.avg:.4f}) '
                .format(
                batch, len(loader_val), batch_time=batch_time,
                loss=losses,
                remain=timeSince(start, float(batch) / len(loader_val)),
            ))

    return losses.avg

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
##
def fit(df, df_kfold):
    skf = StratifiedKFold(n_splits=CFG.num_fold, shuffle=True, random_state=CFG.seed)

    KFOLD = [(idxT,idxV) for i,(idxT,idxV) in enumerate(skf.split(np.arange(df_kfold.shape[0]), df_kfold['stratify_group']))]

    for fold, (idxT, idxV) in enumerate(KFOLD, 1):
        LOGGER.info(f"Training starts ... KFOLD: {fold}/{CFG.num_fold}")

        train = df.loc[idxT, :].reset_index(drop=True)
        val = df.loc[idxV, :].reset_index(drop=True)

        dataset_train = Dataset(df=train, data_dir=CFG.data_dir, transform=transform_train())
        loader_train = DataLoader(dataset_train, batch_size=CFG.batch_size,
                                  sampler=RandomSampler(dataset_train), pin_memory=False, drop_last=True,
                                  num_workers=CFG.num_workers, collate_fn=collate_fn)

        dataset_val = Dataset(df=val, data_dir=CFG.data_dir, transform=transform_val())
        loader_val = DataLoader(dataset_val, batch_size=CFG.batch_size,
                                sampler=SequentialSampler(dataset_val), pin_memory=False, drop_last=True,
                                num_workers=CFG.num_workers, collate_fn=collate_fn)

        config = get_efficientdet_config('tf_efficientdet_d5')
        net = EfficientDet(config, pretrained_backbone=False)
        checkpoint = torch.load('/home/kerrykim/PycharmProjects/3.pothhole_detection/pretrained/efficientdet_d5-ef44aea8.pth')

        # the pretrained weights need to be loaded first,
        # before you modify the head, or you need to modify the state dict to exclude the old head tensors if.
        # you want to do it after.
        config.num_classes = 1
        config.image_size = 512
        net.load_state_dict(checkpoint)
        net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

        net = (DetBenchTrain(net, config)).to(device)

        optim = torch.optim.AdamW(net.parameters(), lr=CFG.lr)

        # if args.use_amp:
        #     model, optimizer = amp.initialize(model, optimizer, opt_level='O1')

        scheduler = ReduceLROnPlateau(optim, **CFG.scheduler_params)

        # scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, args.epochs-1)
        # scheduler = GradualWarmupScheduler(optimizer, multiplier=args.warmup_factor, total_epoch=1, after_scheduler=scheduler_cosine)


        # default value
        st_epoch = 0
        best_loss = 1e20

        for epoch in range(st_epoch + 1, CFG.num_epoch + 1):
            # if epoch < 5:
            #     continue
            start_time = time.time()

            # train
            avg_train_loss = train_one_epoch(loader_train, net, optim, epoch, device)

            # val
            avg_val_loss = val_one_epoch(loader_val, net, device)

            scheduler.step(metrics=avg_val_loss)

            # scoring
            elapsed = time.time() - start_time

            LOGGER.info(
                f'Epoch {epoch} - avg_train_loss: {avg_train_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')

            # save best model
            save_argument = best_loss > avg_val_loss
            best_loss = min(avg_val_loss, best_loss)

            LOGGER.info(f'Epoch {epoch} - Save Best Loss: {best_loss:.4f} Model')

            save_model(ckpt_dir=CFG.ckpt_dir, net=net, num_epoch=CFG.num_epoch, fold=fold,
                       epoch=epoch, batch=CFG.batch_size, save_argument=save_argument)


In [ ]:
##
if __name__ == "__main__":
    seed_everything(CFG.seed)
    df = pd.read_csv(CFG.csv_dir)
    df_kfold = pd.read_csv(CFG.csv2_dir)

    if CFG.mode == 'train':
        torch.set_default_tensor_type('torch.FloatTensor')
        fit(df, df_kfold)